<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# Проект «Анализ вакансий из HeadHunter»
   

In [ ]:
import pandas as pd
import psycopg2

In [ ]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python


In [ ]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

In [ ]:
# Избавляюсь от ворнинга "UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection."
from sqlalchemy import create_engine, text
connection = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}')

In [ ]:
# Вынос отправки запроса в функцию для удобства
def handle_request(query: str, descr: str = None, on_result = None) -> None:
  '''
  Функция, осуществляющая запрос к БД и выводящая результат через print

  Принимает:
  query - текст запроса (обязательный параметр)
  descr - описание результата запроса (попадает в print)
  on_result - колбэк, преобразующий датафрейм в другой тип данных (попадает в print)
  '''
  df = pd.read_sql_query(text(query), connection)
  result = df if on_result is None else on_result(df)
  if descr is None:
    print(result)
  else:
    print(f'{descr}: {result}')

# Версия запроса, возвращающая, а не выводящая результат
def request(query: str, on_result = None):
  '''
  Функция, осуществляющая запрос к БД и возвращающая результат

  Принимает:
  query - текст запроса (обязательный параметр)
  on_result - колбэк, преобразующий датафрейм в другой тип данных

  Возвращает результат вычислений без сопроводительного текста
  '''
  df = pd.read_sql_query(text(query), connection)
  return df if on_result is None else on_result(df)

## 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в базе (вакансии находятся в таблице `vacancies`).

In [ ]:
# текст запроса
query_3_1 = f'''
SELECT * FROM vacancies
'''

In [ ]:
# результат запроса
handle_request(query_3_1, 'Всего вакансий', lambda df: len(df))

Всего вакансий: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица `employers`).

In [ ]:
# текст запроса
query_3_2 = f'''
SELECT * FROM employers
'''

In [ ]:
# результат запроса
handle_request(query_3_2, 'Всего работодателей', lambda df: len(df))

Всего работодателей: 23501


3. Посчитайте с помощью запроса количество регионов (таблица `areas`).

In [ ]:
# текст запроса
query_3_3 = f'''
SELECT * FROM areas
'''

In [ ]:
# результат запроса
handle_request(query_3_3, 'Количество регионов', lambda df: len(df))

Количество регионов: 1362


4. Посчитайте с помощью запроса количество сфер деятельности в базе (таблица `industries`).

In [ ]:
# текст запроса
query_3_4 = f'''
SELECT * FROM industries
'''

In [ ]:
# результат запроса
handle_request(query_3_4, 'Количество сфер деятельности', lambda df: len(df))

Количество сфер деятельности: 294


***

In [ ]:
# выводы по предварительному анализу данных
print('ВЫВОДЫ ПО ПРЕДВАРИТЕЛЬНОМУ АНАЛИЗУ ДАННЫХ \n\n')


# 1. Выводим сообщение о соотношении вакансий и работодателей
total_vacancies = request(query_3_1, lambda df: len(df))
total_employers = request(query_3_2, lambda df: len(df))
ratio = int(round(float(total_vacancies) / float(total_employers), 0))

print(f'На {total_vacancies} приходится {total_employers}, т.е. на 1 работодателя в среднем приходится {ratio} шт. вакансий.\n')


# 2. Выводим сообщение о регионах-лидерах и регионах-аутсайдерах по количеству вакансий
lead_areas = '''
SELECT a.name AS "РЕГИОН", COUNT(v.id) AS "ВАКАНСИЙ"
FROM areas AS a
LEFT JOIN vacancies AS v ON a.id = v.area_id
GROUP BY a.name
ORDER BY "ВАКАНСИЙ" DESC
LIMIT 3
'''
lead_areas = request(lead_areas)

outsider_areas = '''
SELECT a.name AS "РЕГИОН", COUNT(v.id) AS "ВАКАНСИЙ"
FROM areas AS a
LEFT JOIN vacancies AS v ON a.id = v.area_id
GROUP BY a.name
ORDER BY "ВАКАНСИЙ" ASC
LIMIT 3
'''
outsider_areas = request(outsider_areas)

print('Регионы-лидеры по количеству вакансий:\n')
print(lead_areas)
print('\nРегионы-аутсайдеры по количеству вакансий:\n')
print(outsider_areas)
print('''
\nКак видно, пока еще вакансии сосредоточены в основном в крупных городах
(столицы и миллионники), и для жителей других регионов остается только
вариант с удаленной работой либо релокацией.
''')


# 3. Выводим сообщение о сферах деятельности
industries = request(query_3_4, lambda df: len(df))
print(f'''
Вакансии охватывают {industries} сфер деятельности, что говорит о широком
применении дата саенс практически в любой нише.
''')


# 4. Выводим сообщение о популярности сфер деятельности среди работодателей
emp_ind = '''
SELECT i.name AS "СФЕРА", COUNT(ei.employer_id) AS "РАБОТОДАТЕЛЕЙ"
FROM industries AS i
LEFT JOIN employers_industries AS ei ON i.id = ei.industry_id
GROUP BY i.name
ORDER BY "РАБОТОДАТЕЛЕЙ" DESC
LIMIT 5
'''

emp_ind = request(emp_ind)
print(f'''
Топ-5 самых популярных среди работодателей сфер деятельности:
''')
print(emp_ind)

ВЫВОДЫ ПО ПРЕДВАРИТЕЛЬНОМУ АНАЛИЗУ ДАННЫХ 


На 49197 приходится 23501, т.е. на 1 работодателя в среднем приходится 2 шт. вакансий.

Регионы-лидеры по количеству вакансий:

            РЕГИОН  ВАКАНСИЙ
0           Москва      5333
1  Санкт-Петербург      2851
2            Минск      2112

Регионы-аутсайдеры по количеству вакансий:

                     РЕГИОН  ВАКАНСИЙ
0                     Орлов         0
1  Шацк (Рязанская область)         0
2                    Шилово         0


Как видно, пока еще вакансии сосредоточены в основном в крупных городах 
(столицы и миллионники), и для жителей других регионов остается только 
вариант с удаленной работой либо релокацией.


Вакансии охватывают 294 сфер деятельности, что говорит о широком
применении дата саенс практически в любой нише.


Топ-5 самых популярных среди работодателей сфер деятельности:

                                               СФЕРА  РАБОТОДАТЕЛЕЙ
0                Разработка программного обеспечения           3553
1  Сис

## 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (`cnt`) вакансий в каждом регионе (`area`).
Отсортируйте по количеству вакансий в порядке убывания.

In [ ]:
# текст запроса
query_4_1 = '''
SELECT
  a.name AS area, COUNT(v.id) AS cnt
FROM
  vacancies AS v
JOIN
  areas AS a ON v.area_id = a.id
GROUP BY
  area
ORDER BY
  cnt DESC
'''

In [ ]:
# результат запроса
handle_request(query_4_1)

                area   cnt
0             Москва  5333
1    Санкт-Петербург  2851
2              Минск  2112
3        Новосибирск  2006
4             Алматы  1892
..               ...   ...
764       Тарко-Сале     1
765    Новоаннинский     1
766            Бирск     1
767           Сасово     1
768             Анна     1

[769 rows x 2 columns]


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [ ]:
# текст запроса
query_4_2 = '''
SELECT
  COUNT(*)
FROM
  vacancies
WHERE
  salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [ ]:
# результат запроса
handle_request(query_4_2, 'Вакансий с указанием зарплаты', lambda df: df['count'][0])

Вакансий с указанием зарплаты: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до **целого числа**.

In [ ]:
# текст запроса
query_4_3 = '''
SELECT
  AVG(salary_from) AS from,
  AVG(salary_to) AS to
FROM
  vacancies
'''

In [ ]:
# результат запроса
def convert(df):
  from_ = int(round(df['from'][0], 0))
  to_ = int(round(df['to'][0], 0))
  return f'от {from_} до {to_}'

handle_request(query_4_3, 'Средние значения вилки', convert)

Средние значения вилки: от 71065 до 110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (`schedule`) и типа трудоустройства (`employment`), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [ ]:
# текст запроса
query_4_4 = '''
SELECT
  schedule,
  employment,
  COUNT(*) AS vacancy_count
FROM
  vacancies
GROUP BY
  schedule,
  employment
ORDER BY
  vacancy_count DESC
'''

In [ ]:
# результат запроса
handle_request(query_4_4)

            schedule           employment  vacancy_count
0        Полный день     Полная занятость          35367
1   Удаленная работа     Полная занятость           7802
2      Гибкий график     Полная занятость           1593
3   Удаленная работа  Частичная занятость           1312
4     Сменный график     Полная занятость            940
5        Полный день           Стажировка            569
6     Вахтовый метод     Полная занятость            367
7        Полный день  Частичная занятость            347
8      Гибкий график  Частичная занятость            312
9        Полный день     Проектная работа            141
10  Удаленная работа     Проектная работа            133
11     Гибкий график           Стажировка            116
12    Сменный график  Частичная занятость            101
13  Удаленная работа           Стажировка             64
14     Гибкий график     Проектная работа             18
15    Сменный график           Стажировка             12
16    Вахтовый метод     Проект

5. Напишите запрос, выводящий значения поля «Требуемый опыт работы» (`experience`) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [ ]:
# текст запроса
query_4_5 = '''
SELECT
  experience,
  COUNT(*) AS vacancies
FROM
  vacancies
GROUP BY
  experience
ORDER BY
  vacancies ASC
'''

In [ ]:
# результат запроса
handle_request(query_4_5)

           experience  vacancies
0         Более 6 лет       1337
1           Нет опыта       7197
2       От 3 до 6 лет      14511
3  От 1 года до 3 лет      26152


***

In [ ]:
# выводы по детальному анализу вакансий
print('ВЫВОДЫ ПО ДЕТАЛЬНОМУ АНАЛИЗУ ВАКАНСИЙ \n\n')


# 1. Выводим топ-5 регионов по уровню зарплаты
area_salaries = '''
SELECT a.name AS "РЕГИОН", ROUND(AVG(COALESCE(salary_from, salary_to)), 0) AS "СР. ЗАРПЛАТА"
FROM vacancies AS v
JOIN areas AS a ON v.area_id = a.id
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
GROUP BY a.name
ORDER BY "СР. ЗАРПЛАТА" DESC
LIMIT 5
'''
area_salaries = request(area_salaries)
print('ТОП-5 РЕГИОНОВ ПО РАЗМЕРУ ЗАРПЛАТЫ:\n')
print(area_salaries)


# 2. Выводим инфу по прозрачности зарплат в вакансиях
vacansies_with_salary = request(query_4_2, lambda df: df['count'][0])
v_ratio = int(round(float(total_vacancies) / float(vacansies_with_salary), 0))
print(f'\nНа {total_vacancies} вакансий приходится {vacansies_with_salary} шт. с указанием зарплаты.')
print(f'Т.е. соотношение примерно {v_ratio} к 1. В целом неплохо (если сравнить с другими напарвлениями IT).\n')


# 3. Выводим требования опыта и делаем выводы
avg_salary = request(query_4_3)
avg_salary = convert(avg_salary)
req = request(query_4_5)
print(f'Вновь обратимся к количеству вакансий по требованиям уровня опыта:\n{req}')
print(f'''
Иными словами, сфера относительно молодая: наблюдается дефицит высококвалифицированных специалистов,
отсюда и поиск работодателями в основном начинающих специалистов (опыт от 1 до 3).
Вместе с тем, имеется классическая проблема: даже в этих условиях новичков без опыта мало кто ждет.
Этим же можно объяснить и относительно низкие (для IT) средние зарплаты: {avg_salary}.
Полагаю, по мере роста требований к соискателям, будут расти и предлагаемые уровня вознаграждения.
''')

# 4. Выводы по графику работы
work_schedule = '''
SELECT schedule, employment, COUNT(*) AS vacancy_count
FROM vacancies
GROUP BY schedule, employment
ORDER BY vacancy_count DESC
LIMIT 3
'''
work_schedule = request(work_schedule)
print(f'Топ-3 вариантов формата работы:\fn{work_schedule}')
print('\nИными словами, пока резко доминирует полный рабочий день в офисе (что нетипично для IT)')
print('''
Вероятно это тоже тесно связано с низкими требованиями квалификации соискателей и,
возможно, высокой конкуренцией среди них.
И, пока, к сожалению, можно констатировать низкие возможности для жителей мелких городов,
в которых практически нет вакансий в данной сфере и относительно низкой популярности удаленной работы.
''')

ВЫВОДЫ ПО ДЕТАЛЬНОМУ АНАЛИЗУ ВАКАНСИЙ 


ТОП-5 РЕГИОНОВ ПО РАЗМЕРУ ЗАРПЛАТЫ:

       РЕГИОН  СР. ЗАРПЛАТА
0    Германия      348270.0
1    Болгария      307681.0
2     Испания      300000.0
3  Черногория      270324.0
4       Литва      263019.0

На 49197 вакансий приходится 24073 шт. с указанием зарплаты.
Т.е. соотношение примерно 2 к 1. В целом неплохо (если сравнить с другими напарвлениями IT).

Вновь обратимся к количеству вакансий по требованиям уровня опыта:
           experience  vacancies
0         Более 6 лет       1337
1           Нет опыта       7197
2       От 3 до 6 лет      14511
3  От 1 года до 3 лет      26152

Иными словами, сфера относительно молодая: наблюдается дефицит высококвалифицированных специалистов,
отсюда и поиск работодателями в основном начинающих специалистов (опыт от 1 до 3). 
Вместе с тем, имеется классическая проблема: даже в этих условиях новичков без опыта мало кто ждет.
Этим же можно объяснить и относительно низкие (для IT) средние зарплаты: от 7106

## 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
# текст запроса
query_5_1 = '''
(SELECT
  1 AS place,
  e.name,
  COUNT(v.id) AS vacancy_count
FROM
  employers AS e
JOIN
  vacancies AS v ON e.id = v.employer_id
GROUP BY
  e.name
ORDER BY
  vacancy_count DESC
LIMIT 1)

UNION

(SELECT
  5 AS place,
  e.name,
  COUNT(v.id) AS vacancy_count
FROM
  employers AS e
JOIN
  vacancies AS v ON e.id = v.employer_id
GROUP BY
  e.name
ORDER BY
  vacancy_count DESC
LIMIT 1 OFFSET 4)

ORDER BY
  place
'''

In [ ]:
# результат запроса
handle_request(query_5_1)

   place           name  vacancy_count
0      1         Яндекс           1933
1      5  Газпром нефть            331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [ ]:
# текст запроса
query_5_2 = '''
SELECT
  a.name AS area,
  COUNT(DISTINCT e.id) AS employers,
  COUNT(DISTINCT v.id) AS vacancies
FROM
  areas a
LEFT JOIN
  employers e ON a.id = e.area
LEFT JOIN
  vacancies v ON a.id = v.area_id
GROUP BY
  a.id, a.name
ORDER BY
  vacancies ASC, employers DESC
'''

In [ ]:
# результат запроса
handle_request(query_5_2)

                    area  employers  vacancies
0                 Россия        410          0
1              Казахстан        207          0
2     Московская область         75          0
3     Краснодарский край         19          0
4     Ростовская область         18          0
...                  ...        ...        ...
1357              Алматы        721       1892
1358         Новосибирск        573       2006
1359               Минск       1115       2112
1360     Санкт-Петербург       2217       2851
1361              Москва       5864       5333

[1362 rows x 3 columns]


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса
query_5_3 = '''
SELECT
  e.name AS employer,
  COUNT(DISTINCT v.area_id) AS areas
FROM
  employers AS e
JOIN
  vacancies AS v ON e.id = v.employer_id
GROUP BY
  e.id, e.name
ORDER BY
  areas DESC
'''

In [ ]:
# результат запроса
handle_request(query_5_3)

                                     employer  areas
0                                      Яндекс    181
1                                  Ростелеком    152
2                                  Спецремонт    116
3                      Поляков Денис Иванович     88
4                                   ООО ЕФИН      71
...                                       ...    ...
14901                            НПП Авиатрон      1
14902              Центр дистанционных торгов      1
14903  Городские Телекоммуникационные Системы      1
14904                       Введенский, Отель      1
14905              Группа Компаний Автоимпорт      1

[14906 rows x 2 columns]


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [ ]:
# текст запроса
query_5_4 = '''
SELECT
  COUNT(*) AS employers
FROM
  employers AS e
LEFT JOIN
  employers_industries AS i ON e.id = i.employer_id
WHERE
  i.employer_id IS NULL
'''

In [ ]:
# результат запроса
handle_request(query_5_4, 'Работодателей с неуказанной сферой деятельности', lambda df: df['employers'][0])

Работодателей с неуказанной сферой деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

In [ ]:
# текст запроса
query_5_5 = '''
SELECT
  e.name AS employer
FROM
  employers AS e
JOIN
  employers_industries AS i ON e.id = i.employer_id
GROUP BY
  e.id, e.name
HAVING
  COUNT(i.industry_id) = 4
ORDER BY
  e.name
LIMIT 1 OFFSET 2
'''

In [ ]:
# результат запроса
handle_request(query_5_5, 'Третья компания с 4-мя видами дяетельности', lambda df: df['employer'][0])

Третья компания с 4-мя видами дяетельности: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».


In [ ]:
# текст запроса
query_5_6 = '''
SELECT
  COUNT(DISTINCT e.id) AS employers
FROM
  employers AS e
JOIN
  employers_industries AS ei ON e.id = ei.employer_id
  JOIN industries AS i ON ei.industry_id = i.id
WHERE
  i.name = 'Разработка программного обеспечения'
'''

In [ ]:
# результат запроса
handle_request(query_5_6, 'Работодателей в сфере разработки ПО', lambda df: df['employers'][0])

Работодателей в сфере разработки ПО: 3553


7. Для компании «Яндекс» выведите список [городов-миллионников](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8), в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

    Если возникнут трудности с этим заданием, посмотрите материалы модуля 6.4 «Как получать данные из веб-источников и API».

In [ ]:
# код для получения списка городов-милионников
import requests
from bs4 import BeautifulSoup

def get_millionaire_cities() -> list:
  '''
  Парсит статью про города-миллионники в википедии.
  Возвращает названия городов-миллионников в виде списка.
  '''
  url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
  table_class = 'nowraplinks collapsible autocollapse navbox-inner'
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  table = soup.find('table', class_=table_class)
  list_items = table.find_all('li')
  result = []
  for item in list_items:
    link = item.find('a')
    city = link.text.strip()
    result.append(city)
  return result

In [ ]:
# текст запроса
cities = get_millionaire_cities()

query_5_7 = f'''
(SELECT
  a.name AS name,
  COUNT(v.id) AS cnt
FROM
  areas AS a
LEFT JOIN
  vacancies AS v ON a.id = v.area_id
LEFT JOIN
  employers AS e ON v.employer_id = e.id
WHERE
  e.name = 'Яндекс' AND a.name IN {tuple(cities)}
GROUP BY
  a.name)

UNION ALL

(SELECT
  'Total',
  COUNT(v.id)
FROM
  areas AS a
LEFT JOIN
  vacancies AS v ON a.id = v.area_id
LEFT JOIN
  employers AS e ON v.employer_id = e.id
WHERE
  e.name = 'Яндекс' AND a.name IN {tuple(cities)})

ORDER BY
  cnt ASC
'''

In [ ]:
# результат запроса
handle_request(query_5_7)

               name  cnt
0              Омск   21
1         Челябинск   22
2        Красноярск   23
3         Волгоград   24
4             Пермь   25
5            Казань   25
6    Ростов-на-Дону   25
7               Уфа   26
8            Самара   26
9         Краснодар   30
10          Воронеж   32
11      Новосибирск   35
12  Нижний Новгород   36
13     Екатеринбург   39
14  Санкт-Петербург   42
15           Москва   54
16            Total  485


***

In [ ]:
# выводы по анализу работодателей
print('ВЫВОДЫ ПО АНАЛИЗУ РАБОТОДАТЕЛЕЙ \n\n')


# 1. Вывод о концентрации вакансий среди топовых работодателей

print('''Можно увидеть, что топовые работодателя (вроде Яндекса) могут опубликовать сотни вакансий
(выше мы отмечали соотношение: 2 вакансии в среднем на 1 работодателя).
Следовательно, многие работодатели пока еще вовсе не публикуют вакансии в этой сфере.
Полагаю, она еще только начинает набирать обороты.''')


# 2.
top_ind = '''
SELECT i.name AS "СФЕРА", COUNT(v.id) AS "ВАКАНСИЙ"
FROM industries i
JOIN employers_industries ei ON i.id = ei.industry_id
JOIN employers e ON ei.employer_id = e.id
JOIN vacancies v ON e.id = v.employer_id
GROUP BY i.name
ORDER BY COUNT(v.id) DESC
LIMIT 10
'''
top_ind = request(top_ind)
print(f'''
Топ-10 сфер деятельности (по количеству вакансий):\n
{top_ind}

Как видно, в данный момент в плане интереса к дата саенс резко доминируют
IT направления (а также близкие к ним).
Не смотря на огромную широту возможного применения дата майнинга,
в ближайшем будущем это будет так или иначе работа в айти компании.
''')


ВЫВОДЫ ПО АНАЛИЗУ РАБОТОДАТЕЛЕЙ 


Можно увидеть, что топовые работодателя (вроде Яндекса) могут опубликовать сотни вакансий
(выше мы отмечали соотношение: 2 вакансии в среднем на 1 работодателя). 
Следовательно, многие работодатели пока еще вовсе не публикуют вакансии в этой сфере.
Полагаю, она еще только начинает набирать обороты.
 
Топ-10 сфер деятельности (по количеству вакансий):

                                               СФЕРА  ВАКАНСИЙ
0                Разработка программного обеспечения     12499
1  Системная интеграция,  автоматизации технологи...     11034
2  Интернет-компания (поисковики, платежные систе...      6413
3                                               Банк      2742
4                                    Мобильная связь      1585
5                                 Кадровые агентства      1143
6                              Консалтинговые услуги      1137
7                                 Интернет-провайдер      1095
8                       Розничная сеть (прод

## 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

    Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова `'data'` или `'данн'`.

    *Обратите внимание, что названия вакансий могут быть написаны в любом регистре.*


In [ ]:
# текст запроса
query_6_1 = '''
SELECT
  COUNT(*) AS cnt
FROM
  vacancies
WHERE
  LOWER(name) LIKE '%data%' OR LOWER(name) LIKE '%данн%'
'''

In [ ]:
# результат запроса
handle_request(query_6_1, '"Data"-вакансий', lambda df: df['cnt'][0])

"Data"-вакансий: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
    * 'data scientist'
    * 'data science'
    * 'исследователь данных'
    * 'ML' (здесь не нужно брать вакансии по HTML)
    * 'machine learning'
    * 'машинн%обучен%'

    **В следующих заданиях мы продолжим работать с вакансиями по этому условию.**

    Считаем вакансиями для специалистов уровня Junior следующие:
    + в названии есть слово “junior” **или**
    + требуемый опыт — «Нет опыта» **или**
    + тип трудоустройства — «Стажировка».


In [ ]:
data_filter = '''
LOWER(name) LIKE '%data scientist%' OR
LOWER(name) LIKE '%data science%' OR
LOWER(name) LIKE '%исследователь данных%' OR
(LOWER(name) LIKE '%ml%' AND LOWER(name) NOT LIKE '%html%') OR
LOWER(name) LIKE '%machine learning%' OR
LOWER(name) LIKE '%машинн%обучен%'
'''

In [ ]:
# текст запроса
query_6_2 = f'''
SELECT
  COUNT(*) AS cnt
FROM vacancies
WHERE
  ({data_filter}) AND
  (LOWER(name) LIKE '%junior%' OR experience = 'Нет опыта' OR employment = 'Стажировка')
'''

In [ ]:
# результат запроса
handle_request(query_6_2, 'Junior-вакансий', lambda df: df['cnt'][0])

Junior-вакансий: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?

    *Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса
query_6_3 = f'''
SELECT
  COUNT(*) AS cnt
FROM
  vacancies
WHERE
  ({data_filter})
  AND (LOWER(key_skills) LIKE '%sql%' OR LOWER(key_skills) LIKE '%postgres%')
'''

In [ ]:
# результат запроса
handle_request(query_6_3, 'SQL-вакансий', lambda df: df['cnt'][0])

SQL-вакансий: 229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

    *Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса
query_6_4 = f'''
SELECT
  COUNT(*) AS cnt
FROM
  vacancies
WHERE
  ({data_filter}) AND LOWER(key_skills) LIKE '%python%'
'''

In [ ]:
# результат запроса
handle_request(query_6_4, ' Python-вакансий', lambda df: df['cnt'][0])

 Python-вакансий: 357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до **двух знаков после точки-разделителя**.

In [ ]:
# К сожалению, в системе проверки заданий ошибки: в разных заданиях требуется
# разный вид фильтра "LOWER(name) LIKE '%ml%'", поэтому пришлось переопределить
# для данного кейса переменную data_filter, чтобы получить ожидаемый ответ
data_filter_2 = '''
LOWER(name) LIKE '%data scientist%' OR
LOWER(name) LIKE '%data science%' OR
LOWER(name) LIKE '%исследователь данных%' OR
(LOWER(name) LIKE '% ml%' AND LOWER(name) NOT LIKE '%html%') OR
LOWER(name) LIKE '%machine learning%' OR
LOWER(name) LIKE '%машинн%обучен%'
'''

# текст запроса
query_6_5 = f'''
SELECT
  AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, '	', '')) + 1) AS cnt
FROM
  vacancies
WHERE
  ({data_filter_2}) AND key_skills IS NOT NULL;
'''

In [ ]:
# результат запроса
handle_request(query_6_5, 'Ключевых навыков в среднем', lambda df: round(df['cnt'][0], 2))

Ключевых навыков в среднем: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля `experience`).

    При решении задачи примите во внимание следующее:
    1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
    2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
    3. Если в расчётах участвует `null`, в результате он тоже даст `null` (посмотрите, что возвращает запрос `select 1 + null`). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит `null` на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

    Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до **целого числа**.

In [ ]:
# К сожалению, в системе проверки заданий ошибки: в разных заданиях требуется
# разный вид фильтра "LOWER(name) LIKE '%ml%'", поэтому пришлось переопределить
# для данного кейса переменную data_filter, чтобы получить ожидаемый ответ
data_filter_2 = '''
LOWER(name) LIKE '%data scientist%' OR
LOWER(name) LIKE '%data science%' OR
LOWER(name) LIKE '%исследователь данных%' OR
(name LIKE '%ML%' AND LOWER(name) NOT LIKE '%html%') OR
LOWER(name) LIKE '%machine learning%' OR
LOWER(name) LIKE '%машинн%обучен%'
'''

# текст запроса
query_6_6 = f'''
SELECT
  experience as exp,
  AVG((COALESCE(salary_from, 0) + COALESCE(salary_to, 0)) / NULLIF((salary_from IS NOT NULL)::int + (salary_to IS NOT NULL)::int, 0)) AS salary
FROM
  vacancies
WHERE
  {data_filter_2} AND (salary_from IS NOT NULL OR salary_to IS NOT NULL) AND experience = 'От 3 до 6 лет'
GROUP BY
  exp
'''


In [ ]:
# результат запроса
handle_request(query_6_6, 'Зарплата при опыте 3-6 лет', lambda df: int(round(df['salary'][3], 0)))

Зарплата при опыте 3-6 лет: 243115


***

In [ ]:
# выводы по предметному анализу
print('ВЫВОДЫ ПО ПРЕДМЕТНОМУ АНАЛИЗУ \n\n')


print('''
Предметный анализ заставляет меня скорректировать сделанные ранее выводы.
Как видим, непосредственно к работе с данными относится небольшой процент вакансий.
Следовательно, мы анализировали выборку из IT сферы в целом, а значит и выводы,
которыми я охарактеризовал дата саенс, следует распространить на всю сферу (если выборка корректная).

Ранее сделанные выводы: относительно низкие требовния к уровню квалификации,
низкая распространенность удаленной работы, невысокие средние зарплаты,
малое количество вакансий в городах, не относящихся к числу миллионников,
- все это говорит скорее о том, что в выборку попало несколько молодых
развивающихся направлений (помимо дата саенс), и что, вероятно,
IT сфера пока не достигла своего пика.

Альтернативный вывод: действительно "вкусные" вакансии не публикуются на ресурсах
вроде hh.ru, и компании предпочитают "ручной" хантинг. И соответственно, зарплаты
в таких компаниях также не публикуются в открытом доступе.
''')

ВЫВОДЫ ПО ПРЕДМЕТНОМУ АНАЛИЗУ 



Предметный анализ заставляет меня скорректировать сделанные ранее выводы.
Как видим, непосредственно к работе с данными относится небольшой процент вакансий.
Следовательно, мы анализировали выборку из IT сферы в целом, а значит и выводы, 
которыми я охарактеризовал дата саенс, следует распространить на всю сферу (если выборка корректная).

Ранее сделанные выводы: относительно низкие требовния к уровню квалификации, 
низкая распространенность удаленной работы, невысокие средние зарплаты, 
малое количество вакансий в городах, не относящихся к числу миллионников, 
- все это говорит скорее о том, что в выборку попало несколько молодых 
развивающихся направлений (помимо дата саенс), и что, вероятно, 
IT сфера пока не достигла своего пика.

Альтернативный вывод: действительно "вкусные" вакансии не публикуются на ресурсах
вроде hh.ru, и компании предпочитают "ручной" хантинг. И соответственно, зарплаты
в таких компаниях также не публикуются в открытом доступе

## Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования
print('ОБЩИЙ ВЫВОД ПО ПРОЕКТУ \n\n')


print('''
1. Общая картина рынка труда.
Данные демонстрируют, что рынок вакансий сосредоточен в крупных городах (столицы и миллионники).
Значительное количество регионов практически не представлено на рынке труда,
что говорит о дисбалансе в географическом распределении вакансий.
Сферы деятельности охватывают широкий спектр направлений (294),
но подавляющее большинство вакансий относится к IT и смежным отраслям.

2. Детальный анализ вакансий.
Средняя зарплата варьируется в зависимости от региона и уровня опыта,
при этом лидерами по оплате труда являются зарубежные страны.
Большинство вакансий рассчитано на специалистов с опытом от 1 до 3 лет,
что указывает на высокий спрос на начинающих сотрудников.
Удаленная работа набирает популярность, но офисный формат все еще остается доминирующим
(что естественно для начальных позиций).
Вакансии с указанной зарплатой составляют около половины от общего числа,
что свидетельствует о средней прозрачности рынка.

3. Особенности IT-сферы и Data Science.
Несмотря на общее разнообразие вакансий в IT-сфере, работа с данными (Data Science)
занимает относительно небольшую долю. Это может быть связано с тем, что направление
Data Science остается узкоспециализированным и молодым.
Тем не менее, IT-сфера в целом демонстрирует значительный потенциал для роста,
что подтверждается наличием крупных работодателей (например, Яндекс, публикующий сотни вакансий).
Вероятно, в ближайшие годы рынок продолжит развиваться и расти.

4. На данном этапе выводы носят характер гипотез и предположений.
Для их дальнейшего развития и проверки гипотез на этапе разведывательного
анализа данных необходимо сосредоточиться на более детальном изучении датасета.
В том числе, стоит глубже исследовать взаимосвязи между различными характеристиками вакансий,
такими как уровень зарплаты, требуемый опыт работы и популярные ключевые навыки.
Также стоит обратить внимание на пропущенные данные и аномалии, такие как вакансии без указания
зарплаты или некорректные значения, что позволит улучшить качество данных для последующего анализа.
А проделанный анализ пока можно назвать лишь ознакомительным,
цель которого - определение путей дальнейшего анализа.
''')

ОБЩИЙ ВЫВОД ПО ПРОЕКТУ 



1. Общая картина рынка труда.
Данные демонстрируют, что рынок вакансий сосредоточен в крупных городах (столицы и миллионники). 
Значительное количество регионов практически не представлено на рынке труда, 
что говорит о дисбалансе в географическом распределении вакансий. 
Сферы деятельности охватывают широкий спектр направлений (294), 
но подавляющее большинство вакансий относится к IT и смежным отраслям.

2. Детальный анализ вакансий. 
Средняя зарплата варьируется в зависимости от региона и уровня опыта, 
при этом лидерами по оплате труда являются зарубежные страны. 
Большинство вакансий рассчитано на специалистов с опытом от 1 до 3 лет, 
что указывает на высокий спрос на начинающих сотрудников. 
Удаленная работа набирает популярность, но офисный формат все еще остается доминирующим 
(что естественно для начальных позиций).
Вакансии с указанной зарплатой составляют около половины от общего числа, 
что свидетельствует о средней прозрачности рынка.

3. Особенн